In [ ]:
### Common Source Code
from datetime import datetime, timedelta

import json

import re

_topic_cache = {}

_TONE_TEMPLATES = {
    "neutral":           "Write a factual, objective summary of the topic.",
    "informative":       "Explain the topic clearly and concisely, focusing on key facts.",
    "serious":           "Discuss the topic in a respectful, serious tone highlighting importance.",
    "playful":           "Write in a lighthearted, playful voice, using fun wordplay.",
    "humorous":          "Craft a witty, amusing one-liner.",
    "witty":             "Compose a sharp, clever one-liner.",
    "empathetic":        "Acknowledge feelings with empathy and support.",
    "inspirational":     "Create an uplifting, motivational message.",
    "motivational":      "Write a caption that energizes and prompts action.",
    "supportive":        "Offer reassurance and encouragement.",
    "suspenseful":       "Build curiosity with a suspenseful teaser.",
    "reflective":        "Invite thoughtful introspection.",
    "poetic":            "Use vivid imagery and metaphor in lyrical form.",
    "analytical":        "Provide an analytical breakdown with key insights.",
    "critical":          "Present a balanced critique, noting pros and cons.",
    "descriptive":       "Paint a vivid picture with sensory details.",
    "conversational":    "Write casually, as if speaking with a friend.",
    "authoritative":     "Demonstrate expertise with confident tone.",
    "educational":       "Simplify complex ideas into clear educational language.",
    "promotional":       "Highlight benefits and encourage engagement.",
    "nostalgic":         "Evoke fond memories and a sense of nostalgia.",
    "inquisitive":       "Pose an engaging question to invite responses.",
    "urgent":            "Write a call-to-action prompting immediate attention.",
    "cautionary":        "Warn about risks or pitfalls in a cautionary style.",
    "celebratory":       "Express enthusiasm and celebration.",
    "sarcastic":         "Convey the message with ironic, tongue-in-cheek humor.",
    "optimistic":        "Focus on positive outcomes and hopeful possibilities.",
    "skeptical":         "Question assumptions and express healthy doubt.",
    "minimalist":        "Say as much as possible with as few words as necessary.",
    "dramatic":          "Heighten stakes and emotion with theatrical flair.",
    "mysterious":        "Hint at more beneath the surface; keep it intriguing and slightly vague.",
    "friendly":          "Sound warm, approachable, and casual like a trusted peer.",
    "empowering":        "Give the audience a sense of agency and strength.",
    "edgy":              "Take a bold, slightly provocative stance to stand out.",
    "exclusive":         "Make the audience feel like they're getting insider access.",
    "trendsetting":      "Frame it as ahead-of-the-curve and culturally leading.",
    "reassuring":        "Calm concerns and reinforce trust gently.",
    "confident":         "State points decisively with no hesitation.",
    "behind-the-scenes": "Reveal insider context in a candid, informal way.",
}

_ALLOWED_GENRES = [
    "news", "breaking_news", "entertainment", "movies", "music", "sports", "tech",
    "artificial_intelligence", "lifestyle", "health", "wellness", "fitness",
    "business", "startups", "finance", "personal_finance", "crypto", "real_estate",
    "arts", "culture", "community", "education", "science", "environment",
    "sustainability", "travel", "food", "gaming", "fashion", "beauty",
    "relationships", "parenting", "self_help", "spirituality", "automotive",
    "DIY", "pets", "career", "politics", "policy", "local", "global", "niche",
    "events", "celebrity", "trends", "memes", "innovation", "social_justice"
]

_ALLOWED_FORMATS = [
    "listicle", "how-to", "tutorial", "question", "trivia", "quote", "narrative",
    "story", "thread", "comparison", "reaction", "news-flash", "behind-the-scenes",
    "interview", "faq", "checklist", "myth-busting", "before-after", "survey",
    "poll", "prediction", "countdown", "tips", "case_study", "testimonial",
    "mini-review", "challenge", "announcement", "event_recap", "opinion",
    "analysis", "explainer", "warning", "celebration", "reminder", "spotlight",
    "guide", "hack", "projection", "quote-card"
]

_ALLOWED_AUDIENCES = [
    "general_public", "influencers", "local_community", "entrepreneurs",
    "students", "creators", "early_adopters", "parents", "professionals",
    "investors", "gamers", "travelers", "foodies", "fitness_enthusiasts",
    "fashionistas", "pet_owners", "eco-conscious", "diyers", "researchers",
    "local_businesses", "tourists", "educators", "tech_enthusiasts", "career_seekers",
    "content_marketers", "nonprofit_leaders", "policy_makers", "health_advocates"
]

def _normalize_list(lst):
    seen = set()
    out = []
    for x in lst:
        if not isinstance(x, str):
            continue
        key = x.strip().lower()
        if key and key not in seen:
            seen.add(key)
            out.append(key)
    return out

def _validate_taxonomy(genre, tone, fmt, audience):
    # genre: list, tone: string, fmt: string, audience: list
    validated = {}
    # genres: intersect with allowed, take up to 3
    genres = [g for g in genre if isinstance(g, str)]
    genres = [g.lower() for g in genres]
    genres = [g for g in genres if g in _ALLOWED_GENRES]
    validated["genre"] = genres[:3] if genres else []
    # tone: fallback to neutral if unknown
    t = tone.lower() if isinstance(tone, str) else ""
    validated["tone"] = t if t in _TONE_TEMPLATES else "neutral"
    # format
    f = fmt.lower() if isinstance(fmt, str) else ""
    validated["format"] = f if f in _ALLOWED_FORMATS else "listicle"
    # audience
    auds = [a for a in audience if isinstance(a, str)]
    auds = [a.lower() for a in auds]
    auds = [a for a in auds if a in _ALLOWED_AUDIENCES]
    validated["audience"] = auds[:3] if auds else ["general_public"]
    return validated

def _validate_captions_structure(captions):
    if not isinstance(captions, list):
        return False
    for c in captions:
        if not isinstance(c, str):
            return False
    return True

def safe_parse_json(raw_text, description, client=None):
    try:
        return json.loads(raw_text)
    except json.JSONDecodeError as e:
        if client:
            client.stream_message(f"⚠️ Failed to parse {description} JSON directly: {e}. Attempting recovery…\n")
            client.stream_message(f"Raw output was: {raw_text[:1000]}{'...(truncated)' if len(raw_text) > 1000 else ''}\n")
        # Attempt to extract the first JSON object substring (handles nested braces)
        try:
            # Recursive regex using balancing is not natively supported; simple fallback: find first {...}
            match = re.search(r"\{[^{}]*\}", raw_text)
            if match:
                try:
                    return json.loads(match.group(0))
                except Exception as e2:
                    if client:
                        client.stream_message(f"⚠️ Recovery attempt for {description} also failed: {e2}\n")
        except re.error:
            pass
        return {}



### Workflow Source Code

### <CaptionGen node source code>
def caption_generator(topic):
    """
    Generate context-aware, multi-genre/tone/format/audience Instagram captions for a topic,
    caching with 24h TTL, and defensive coding.
    """
    from abacusai import ApiClient, AgentResponse, Blob

    client = ApiClient()
    now = datetime.utcnow()
    ttl = timedelta(hours=24)

    # 1) Prepare research
    search_queries = [
        f"{topic}"
        f"{topic} latest news",
        f"{topic} trending",
        f"{topic} instagram posts",
    ]

    # 2) Cache lookup
    cache = _topic_cache.get(topic)
    if cache and (now - cache['cached_at'] < ttl):
        client.stream_message(f"♻️ Using cached results (age {(now - cache['cached_at']).total_seconds()/3600:.1f}h)\n")
        genre_list    = cache['genre']
        tone_label    = cache['tone']
        format_label  = cache['format']
        audience_list = cache['audience']
        summary       = cache['summary']
        captions      = cache['captions']
    else:
        # 3) Research
        client.stream_message(f"🔍 Researching topic: {topic}...\n")
        snippets = []
        for q in search_queries:
            try:
                resp = client.search_web_for_llm(queries=[q], fetch_content=True, max_results=2)
                for r in getattr(resp, 'search_results', []):
                    content = getattr(r, 'content', '') or getattr(r, 'snippet', '') or getattr(r, 'title', '')
                    if content:
                        snippets.append(content)
            except Exception as e:
                client.stream_message(f"⚠️ Search error for '{q}': {e}\n")
        combined = ' '.join(snippets[:6]) or f"General information about {topic}"

        # 4) Topic taxonomy classification
        client.stream_message("🗂 Classifying topic taxonomy with expanded options...\n")
        try:
            taxonomy_schema = {
                "genre": {
                    "type": "list",
                    "description": "List of up to 3 genres relevant to the topic.",
                    "is_required": True,
                },
                "tone": {
                    "type": "string",
                    "description": "The primary tonal style to use.",
                    "is_required": True,
                },
                "format": {
                    "type": "string",
                    "description": "The caption format or structure.",
                    "is_required": True,
                },
                "audience": {
                    "type": "list",
                    "description": "Target audience segments.",
                    "is_required": True,
                },
            }

            system_msg = f"""
Classify the topic into the following JSON schema. Use up to three entries for list fields.
Allowed genres: {', '.join(_ALLOWED_GENRES)}.
Allowed tones: any of the keys from the tone templates: {', '.join(_TONE_TEMPLATES.keys())}.
Allowed formats: {', '.join(_ALLOWED_FORMATS)}.
Allowed audiences: {', '.join(_ALLOWED_AUDIENCES)}.
Provide the most relevant items based on the research snippet.
Output ONLY valid JSON with keys: genre, tone, format, audience. Do not include any explanation or extra fields.
"""

            tax_resp = client.evaluate_prompt(
                prompt=f"Research: {combined[:1000]}",
                system_message=system_msg,
                llm_name="GEMINI_2_FLASH",
                response_type="json",
                json_response_schema=taxonomy_schema,
                max_tokens=300
            )
            raw_tax = safe_parse_json(getattr(tax_resp, "content", "") or "", "taxonomy", client)
            validated = _validate_taxonomy(
                genre=raw_tax.get('genre', []),
                tone=raw_tax.get('tone', 'neutral'),
                fmt=raw_tax.get('format', 'listicle'),
                audience=raw_tax.get('audience', [])
            )
            genre_list    = validated['genre']
            tone_label    = validated['tone']
            format_label  = validated['format']
            audience_list = validated['audience']
        except Exception as e:
            client.stream_message(f"⚠️ Taxonomy classification failed: {e}\n")
            # fallback defaults
            genre_list, tone_label, format_label, audience_list = ["niche"], "neutral", "listicle", ["general_public"]

        # Enforce neutrality if politics or policy appears
        if any(g in {"politics", "policy"} for g in genre_list):
            tone_label = "neutral"

        # 5) Summary + captions generation
        client.stream_message("✂️ Generating summary and captions with diversity...\n")
        try:
            summary_captions_schema = {
                "summary": {
                    "type": "string",
                    "description": "A concise summary of the topic suitable for social media captioning.",
                    "is_required": True,
                },
                "captions": {
                    "type": "list",
                    "description": "A list of suggested captions for Instagram posts.",
                    "is_required": True,
                },
            }

            prompt_txt = (
                f"Topic: {topic}\n"
                f"Research summary: {combined[:800]}\n"
                f"Genres: {', '.join(genre_list)}\n"
                f"Tone: {tone_label}\n"
                f"Format: {format_label}\n"
                f"Audience: {', '.join(audience_list)}"
            )
            sys_msg = (
                _TONE_TEMPLATES.get(tone_label, _TONE_TEMPLATES['neutral']) +
                f" Generate **exactly five** unique captions (no more, no fewer) in {format_label} format, tailored for {', '.join(audience_list)}. "
                "Each caption should include relevant hashtags and optionally emojis. Vary phrasing so they feel distinct. "
                "Output ONLY valid JSON with keys 'summary' and 'captions' and no extra commentary."
            )

            def _unique_captions(captions_list):
                seen = set()
                unique = []
                for c in captions_list:
                    if not isinstance(c, str):
                        continue
                    cleaned = c.strip()
                    key = cleaned.lower()
                    if key and key not in seen:
                        seen.add(key)
                        unique.append(cleaned)
                return unique

            # ── NEW: ask Gemini and retry up to 3 times until we have ≥5 captions ──
            def _ask_gemini_once(system_message):
                resp = client.evaluate_prompt(
                    prompt=prompt_txt,
                    system_message=system_message,
                    llm_name="GEMINI_2_FLASH",
                    response_type="json",
                    json_response_schema=summary_captions_schema,
                    max_tokens=700
                )
                return safe_parse_json(getattr(resp, "content", "") or "", "summary+captions", client)

            summary = ""
            captions = []
            for attempt in range(1, 4):  # tries 1-3
                try:
                    data = _ask_gemini_once(sys_msg)
                    summary = data.get('summary', '').strip()
                    raw_captions = data.get('captions', [])
                    captions = _unique_captions(raw_captions)
                    if len(captions) >= 5:
                        captions = captions[:5]
                        break  # success
                    client.stream_message(
                        f"⚠️ Gemini returned {len(captions)} caption(s) (try {attempt}/3). Retrying…\n"
                    )
                except Exception as e:
                    client.stream_message(f"⚠️ Gemini error (try {attempt}/3): {e}\n")
                    # on malformed JSON, optionally retry with stricter reminder
                    if attempt == 1:
                        sys_msg = sys_msg + " AGAIN: Output only the JSON object, no explanation."
            # final safeguard – pad to five if still short
            if len(captions) < 5:
                client.stream_message("⚠️ Using fallback captions to reach five.\n")
                base = f"{topic} is trending—stay tuned!"
                while len(captions) < 5:
                    captions.append(base)
        except Exception as e:
            client.stream_message(f"⚠️ Summary+captions failed: {e}\n")
            summary, captions = f"{topic} is trending.", [f"{topic} is trending—stay tuned!"]

        # 6) Cache results
        _topic_cache[topic] = {
            "genre":    genre_list,
            "tone":     tone_label,
            "format":   format_label,
            "audience": audience_list,
            "summary":  summary,
            "captions": captions,
            "cached_at": now
        }

    # 7) Build and return response
    client.stream_message("📦 Creating output files...\n")
    markdown_lines = [
        f"# Caption Generation Report: {topic}",
        "",
        f"**Genres:** {', '.join(genre_list)}",
        f"**Tone:** {tone_label}",
        f"**Format:** {format_label}",
        f"**Audience:** {', '.join(audience_list)}",
        "",
        "## Summary",
        summary or "(no summary generated)",
        "",
        "## Captions"
    ]
    for idx, cap in enumerate(captions, 1):
        markdown_lines += [f"### Caption {idx}", cap.strip(), ""]
    markdown_lines += [
        "## Research Queries",
        f"{', '.join(search_queries)}"
    ]
    md_content = "\n".join(markdown_lines)
    md_bytes   = md_content.encode("utf-8")
    json_blob  = {
        "topic":     topic,
        "genre":     genre_list,
        "tone":      tone_label,
        "format":    format_label,
        "audience":  audience_list,
        "summary":   summary,
        "captions":  captions,
        "cached_at": now.isoformat()
    }
    json_bytes = json.dumps(json_blob, indent=2).encode("utf-8")

    return AgentResponse(
        summary=summary,
        sentiment=tone_label,
        captions=captions,
        topic=topic,
        markdown_file=Blob(
            contents=md_bytes,
            mime_type="text/markdown",
            filename=f"captiongen_report_{topic.replace(' ', '_')}.md"
        ),
        json_file=Blob(
            contents=json_bytes,
            mime_type="application/json",
            filename=f"captiongen_data_{topic.replace(' ', '_')}.json"
        )
    )

### <ImageGen node source code>
def image_generator_corrected_fixed(captions, sentiment, summary, topic):
    """
    Generate viral Instagram-style images conditioned on individual captions.
    Uses Hugging Face InferenceClient with provider="replicate" to call Stable Diffusion 3.5 Large (and fallbacks).
    One image per caption (5 captions = 5 images total).
    Robust JSON parsing, strict schema enforcement, and retry logic for malformed LLM outputs.
    """
    import json
    import os
    import time
    import re
    from abacusai import ApiClient, AgentResponse, Blob

    # ▸▸ Coerce captions from JSON string if needed
    if isinstance(captions, str):
        try:
            captions = json.loads(captions)
        except Exception:
            pass

    if not isinstance(captions, list) or len(captions) == 0:
        captions = [f"{topic} is trending on Instagram"] * 5

    # Helper for resilient JSON parsing
    def safe_parse_json(raw_text, description, client=None):
        try:
            return json.loads(raw_text)
        except json.JSONDecodeError as e:
            if client:
                client.stream_message(f"⚠️ Failed to parse {description} JSON directly: {e}. Attempting recovery…\n")
                client.stream_message(f"Raw output: {raw_text[:1500]}{'...(truncated)' if len(raw_text) > 1500 else ''}\n")
            candidates = []
            # Simple first {...} block
            m1 = re.search(r"\{[^{}]*\}", raw_text)
            if m1:
                candidates.append(m1.group(0))
            # Balanced-brace heuristic
            stack = []
            start_idx = None
            for i, ch in enumerate(raw_text):
                if ch == "{":
                    if start_idx is None:
                        start_idx = i
                    stack.append("{")
                elif ch == "}":
                    if stack:
                        stack.pop()
                        if not stack and start_idx is not None:
                            candidates.append(raw_text[start_idx : i + 1])
                            break
            for cand in candidates:
                try:
                    return json.loads(cand)
                except Exception as e2:
                    if client:
                        client.stream_message(f"⚠️ Recovery candidate failed for {description}: {e2}\n")
            return {}

    client = ApiClient()
    client.stream_message(f"🚀 Starting caption-conditioned image generation for topic: {topic}\n")

    # ---- Secrets ----
    hf_api_key = None
    try:
        hf_api_key = client.get_organization_secret(secret_key="HUGGINGFACE_API_KEY").value
        client.stream_message("✅ Retrieved Hugging Face API key.\n")
    except Exception as e:
        client.stream_message(f"⚠️ Could not retrieve Hugging Face API key: {e}\n")

    # ---- Sentiment-based style guide ----
    sentiment_styles = {
        "tragic": "dramatic lighting, emotional composition, soft shadows, muted tones, photorealistic",
        "happy": "bright vibrant lighting, joyful atmosphere, golden hour glow, photorealistic",
        "political": "documentary editorial style, clean composition, balanced lighting, professional photorealism",
        "funny": "playful expressions, saturated colors, whimsical props, high energy, photorealistic",
        "pop-culture": "stylish modern aesthetic, trendy filters, influencer vibe, photorealistic",
        "offensive": "informative layout, neutral lighting, respectful tonality, photorealistic",
        "educational": "clean academic aesthetic, professional lighting, campus atmosphere, photorealistic",
        "inspirational": "uplifting composition, golden hour, aspirational mood, photorealistic",
        "analytical": "infographic style elements, data visualization hints, modern tech aesthetic, photorealistic",
        "nostalgic": "warm vintage tones, soft focus edges, memory-like quality, photorealistic",
        "mysterious": "moody lighting, intriguing shadows, cinematic atmosphere, photorealistic",
        "celebratory": "festive colors, dynamic energy, joyful composition, photorealistic"
    }
    style_guide = sentiment_styles.get(
        (sentiment or "").lower(),
        "professional photography, engaging composition, Instagram-optimized, photorealistic",
    )

    # Attempt to import HF client
    try:
        from huggingface_hub import InferenceClient
    except ImportError:
        InferenceClient = None  # fallback to REST

    def call_hf_diffusion(prompt, negative_prompt, guidance_scale, steps):
        primary_model = os.getenv("HF_MODEL_REPO", "stabilityai/stable-diffusion-3.5-large")
        fallbacks = ["stabilityai/sdxl-turbo", "stabilityai/stable-diffusion-xl-base-1.0"]
        tried = []

        if not hf_api_key:
            raise RuntimeError("No Hugging Face API key available for inference call.")

        client.stream_message(f"ℹ️ Primary HF model resolved to '{primary_model}'. Fallbacks: {fallbacks}\n")

        if InferenceClient is not None:
            try:
                hf = InferenceClient(provider="replicate", api_key=hf_api_key, timeout=300)
                client.stream_message(f"🧪 Using HuggingFace InferenceClient (replicate) for model '{primary_model}'\n")
                raw = hf.text_to_image(
                    prompt,
                    model=primary_model,
                    guidance_scale=guidance_scale,
                    num_inference_steps=steps,
                    negative_prompt=negative_prompt,
                )
                # Normalize outputs
                if isinstance(raw, (bytes, bytearray)):
                    return bytes(raw)
                try:
                    from PIL import Image
                    import io
                    if isinstance(raw, Image.Image):
                        buf = io.BytesIO()
                        raw.save(buf, format="PNG")
                        return buf.getvalue()
                except ImportError:
                    pass
                import io
                if isinstance(raw, dict):
                    img = raw.get("image") or raw.get("images") or None
                    if isinstance(img, list) and img:
                        img = img[0]
                    if isinstance(img, (bytes, bytearray)):
                        return bytes(img)
                    try:
                        from PIL import Image
                        if isinstance(img, Image.Image):
                            buf = io.BytesIO()
                            img.save(buf, format="PNG")
                            return buf.getvalue()
                    except Exception:
                        pass
                if isinstance(raw, str):
                    import base64
                    try:
                        return base64.b64decode(raw)
                    except Exception:
                        pass
                client.stream_message(f"⚠️ Unexpected return type from InferenceClient for model '{primary_model}': {type(raw)}. Falling back.\n")
                tried.append((primary_model, f"unexpected return type {type(raw)}"))
            except Exception as e:
                client.stream_message(f"⚠️ InferenceClient (replicate) call failed for '{primary_model}': {e}\n")
                tried.append((primary_model, f"replicate-client error: {e}"))

        # REST fallback
        import requests
        for model in [primary_model] + fallbacks:
            hf_url = f"https://api-inference.huggingface.co/models/{model}"
            headers = {"Authorization": f"Bearer {hf_api_key}"}
            payload = {
                "inputs": prompt,
                "parameters": {
                    "negative_prompt": negative_prompt,
                    "guidance_scale": guidance_scale,
                    "num_inference_steps": steps,
                },
                "options": {"wait_for_model": True},
            }
            try:
                resp = requests.post(hf_url, headers=headers, json=payload, timeout=180)
            except Exception as e:
                client.stream_message(f"⚠️ Network error calling HF model '{model}': {e}\n")
                tried.append((model, f"network error: {e}"))
                continue

            if resp.status_code == 200:
                if model != primary_model:
                    client.stream_message(f"ℹ️ Fell back to model '{model}' after primary '{primary_model}' failed.\n")
                return resp.content

            body_preview = resp.text if len(resp.text) < 1000 else resp.text[:1000] + "...(truncated)"
            client.stream_message(
                f"❌ HF inference failed for model '{model}': status={resp.status_code}, body={body_preview}\n"
            )
            tried.append((model, f"status {resp.status_code}"))

        raise RuntimeError(f"All HF models failed: {tried}")

    # ---- Negative prompt generation with robust parsing ----
    client.stream_message("🧠 Generating global negative prompt...\n")
    default_negative = "blurry, deformed, watermark, low resolution, bad anatomy, oversaturated"
    try:
        neg_schema = {
            "negative_prompt": {
                "type": "string",
                "description": "Negative prompt to avoid unwanted artifacts in image generation",
                "is_required": True
            }
        }
        neg_resp = client.evaluate_prompt(
            prompt=(
                f"Captions: {captions[:3]}\n"
                f"Style Guide: {style_guide}\n"
                "From the above, produce a concise negative prompt that avoids common diffusion artifacts."
            ),
            system_message=(
                "You are an image quality guard. Given the inputs, output ONLY a single valid JSON object "
                "with the key \"negative_prompt\" whose value is a concise string avoiding common diffusion artifacts. "
                "Do not include any explanation, extra keys, or surrounding text. Example format: "
                "{\"negative_prompt\": \"blurry, low-res, watermark\"}."
            ),
            llm_name="GEMINI_2_FLASH",
            response_type="json",
            json_response_schema=neg_schema,
            max_tokens=60,
        )
        neg_data = safe_parse_json(getattr(neg_resp, "content", "") or "", "negative prompt", client)
        negative_prompt = neg_data.get("negative_prompt", default_negative)
        if not isinstance(negative_prompt, str) or not negative_prompt.strip():
            raise ValueError("Invalid negative prompt")
    except Exception as e:
        client.stream_message(f"⚠️ Negative prompt generation failed: {e}. Using default.\n")
        negative_prompt = default_negative

    # ---- Per-caption prompt generation with retry and robust parsing ----
    client.stream_message("🧠 Generating image prompts for five captions...\n")
    caption_prompts = []
    captions5 = (captions + captions[:5])[:5]
    for i, cap in enumerate(captions5, start=1):
        image_prompt = ""
        base_system_msg = (
            "You are a visual prompt engineer for viral Instagram content. Given the caption, topic, sentiment, summary, "
            "and style guide, output ONLY a single valid JSON object with the key \"image_prompt\" whose value is a rich, "
            "diffusion-ready image prompt. Do not include explanation, commentary, markdown, or extra fields. "
            "Example format: {\"image_prompt\": \"Close-up, golden-hour lighting, dynamic angle, enthusiastic mood...\"}. "
            "Include composition, lighting, camera angle, mood, color palette, and a thumbnail-style hook in the prompt."
        )
        prompt_input = (
            f"Caption: {cap}\n"
            f"Topic: {topic}\n"
            f"Sentiment: {sentiment}\n"
            f"Summary: {summary}\n"
            f"Style Guide: {style_guide}\n"
        )

        for attempt in range(1, 3):  # primary + one stricter retry
            try:
                system_msg = base_system_msg
                if attempt > 1:
                    system_msg += " AGAIN: Output only the JSON object, no explanation."
                response = client.evaluate_prompt(
                    prompt=prompt_input,
                    system_message=system_msg,
                    llm_name="GEMINI_2_FLASH",
                    response_type="json",
                    json_response_schema={"image_prompt": {"type": "string", "description": "Visual prompt", "is_required": True}},
                    max_tokens=180,
                )
                raw = getattr(response, "content", "") or ""
                data = safe_parse_json(raw, f"image prompt for caption {i}", client)
                image_prompt = data.get("image_prompt", "").strip()
                if not image_prompt:
                    client.stream_message(f"⚠️ Empty or invalid image_prompt for caption {i} on attempt {attempt}; raw output: {raw[:1500]}{'...(truncated)' if len(raw) > 1500 else ''}\n")
                    raise ValueError("Empty image prompt")
                caption_prompts.append({"caption": cap, "image_prompt": image_prompt})
                client.stream_message(f"✅ Prompt for caption {i} obtained on attempt {attempt}.\n")
                break  # success
            except Exception as e:
                client.stream_message(f"⚠️ Prompt gen fallback attempt {attempt} for caption {i}: {e}\n")
                if attempt == 2:
                    fallback = f"{cap}, {style_guide}, engaging Instagram reel thumbnail, photorealistic"
                    caption_prompts.append({"caption": cap, "image_prompt": fallback})
        # end caption loop

    # ---- Image generation: 1 image per caption ----
    client.stream_message("🖼️ Generating images...\n")
    generated_images = []
    image_data_list = []
    for cp_idx, cp in enumerate(caption_prompts, start=1):
        base_prompt = cp["image_prompt"]
        prompt_text = f"{base_prompt}, trending Instagram reel thumbnail"
        guidance = 7.5
        steps = 30
        success = False
        attempt = 0
        while attempt < 2 and not success:
            attempt += 1
            try:
                client.stream_message(f"→ Generating image for caption {cp_idx} (attempt {attempt})...\n")
                image_bytes = call_hf_diffusion(
                    prompt=prompt_text,
                    negative_prompt=negative_prompt,
                    guidance_scale=guidance,
                    steps=steps,
                )
                filename = f"img_{topic.replace(' ', '_')}_cap{cp_idx}.png"
                generated_images.append(Blob(contents=image_bytes, mime_type="image/png", filename=filename))
                image_data_list.append({
                    "caption_index": cp_idx,
                    "caption": cp["caption"],
                    "prompt": prompt_text,
                    "negative_prompt": negative_prompt,
                    "filename": filename,
                    "backend": "hf_inference",
                    "tuned_params": {"guidance": guidance, "steps": steps},
                })
                client.stream_message(f"✅ Generated {filename}\n")
                success = True
            except Exception as e:
                client.stream_message(f"✗ Error generating image for caption {cp_idx}: {e}\n")
                time.sleep(0.8)
        if not success:
            client.stream_message(f"⚠️ Using fallback for caption {cp_idx}\n")
            mock_png = (
                b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x01"
                b"\x00\x00\x00\x01\x08\x02\x00\x00\x00\x90wS\xde"
                b"\x00\x00\x00\tpHYs\x00\x00\x0b\x13\x00\x00\x00\x0b"
                b"\x13\x01\x00\x9a\x9c\x18\x00\x00\x00\x12IDATx"
                b"\x9cc```\x00\x00\x00\x04\x00\x01\xddCC\x9e\x00"
                b"\x00\x00\x00IEND\xaeB`\x82"
            )
            filename = f"img_{topic.replace(' ', '_')}_cap{cp_idx}_fallback.png"
            generated_images.append(Blob(contents=mock_png, mime_type="image/png", filename=filename))
            image_data_list.append({
                "caption_index": cp_idx,
                "caption": cp["caption"],
                "prompt": prompt_text,
                "negative_prompt": negative_prompt,
                "filename": filename,
                "backend": "mock",
                "tuned_params": {"guidance": guidance, "steps": steps},
            })

    # ---- Report ----
    client.stream_message("📝 Building report...\n")
    markdown_content = f"# Image Generation Report: {topic}\n\n**Sentiment:** {sentiment}\n\n**Summary:** {summary}\n\n**Style Guide:** {style_guide}\n\n"
    markdown_content += "## Caption-conditioned Prompts\n"
    for idx, cp in enumerate(caption_prompts, 1):
        markdown_content += f"### Caption {idx}\n- **Caption:** {cp['caption']}\n- **Image Prompt:** {cp['image_prompt']}\n\n"
    markdown_content += "\n## Generated Images\n"
    for img in image_data_list:
        markdown_content += (
            f"### Caption {img['caption_index']}\n"
            f"- **Filename:** {img['filename']}\n"
            f"- **Prompt:** {img['prompt']}\n"
            f"- **Negative Prompt:** {img['negative_prompt']}\n"
            f"- **Backend:** {img['backend']}\n"
            f"- **Tuned Params:** {img['tuned_params']}\n\n"
        )

    json_data = {
        "topic": topic,
        "sentiment": sentiment,
        "summary": summary,
        "style_guide": style_guide,
        "caption_prompts": caption_prompts,
        "negative_prompt": negative_prompt,
        "generated_images": image_data_list,
        "total_images": len(generated_images),
    }

    markdown_bytes = markdown_content.encode("utf-8")
    json_bytes = json.dumps(json_data, indent=2).encode("utf-8")

    # ---- Return first five images ----
    padded = generated_images[:5]
    while len(padded) < 5:
        pad_idx = len(padded) + 1
        mock_png = (
            b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x01"
            b"\x00\x00\x00\x01\x08\x02\x00\x00\x00\x90wS\xde"
            b"\x00\x00\x00\tpHYs\x00\x00\x0b\x13\x00\x00\x00\x0b"
            b"\x13\x01\x00\x9a\x9c\x18\x00\x00\x00\x12IDATx"
            b"\x9cc```\x00\x00\x00\x04\x00\x01\xddCC\x9e\x00"
            b"\x00\x00\x00IEND\xaeB`\x82"
        )
        filename = f"pad_{topic.replace(' ', '_')}_{pad_idx}.png"
        padded.append(Blob(contents=mock_png, mime_type="image/png", filename=filename))

    client.stream_message(f"✅ Image generation complete! Generated {len(generated_images)} images.\n")

    return AgentResponse(
        image1=padded[0],
        image2=padded[1],
        image3=padded[2],
        image4=padded[3],
        image5=padded[4],
        topic=topic,
        sentiment=sentiment,
        total_images=len(generated_images),
        markdown_file=Blob(
            contents=markdown_bytes,
            mime_type="text/markdown",
            filename=f"imagegen_report_{topic.replace(' ', '_')}.md",
        ),
        json_file=Blob(
            contents=json_bytes,
            mime_type="application/json",
            filename=f"imagegen_data_{topic.replace(' ', '_')}.json",
        ),
    )
